In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
%matplotlib inline
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
pd.set_option('display.max_columns', None)  
def makePrediction(stationNumber):
    #read in databases
    chart1 = requests.get("http://localhost:5000/chartData/"+stationNumber)
    chart2 = requests.get("http://localhost:5000/pastWeather")
    #load as json
    mainJson = json.loads(chart1.text)
    weatherJson=json.loads(chart2.text)

    #flatten the weather json
    dayofyear=[]
    description=[]
    weekday=[]
    hour=[]
    for i in weatherJson['dayofyear']:
        dayofyear.append(weatherJson['dayofyear'][i])

    for i in weatherJson['description']:
        description.append(weatherJson['description'][i])

    for i in weatherJson['weekday']:
        weekday.append(weatherJson['weekday'][i])

    for i in weatherJson['hour']:
        hour.append(weatherJson['hour'][i])
    weatherDict={}
    weatherDict['Dayofyear']= dayofyear
    weatherDict['description'] = description
    weatherDict['hour']=hour

    available_bikes=[]
    mainDayofyear=[]
    weekday=[]
    hour=[]
    for i in mainJson['available_bikes']:
        available_bikes.append(mainJson['available_bikes'][i])

    for i in mainJson['dayofyear']:
        mainDayofyear.append(mainJson['dayofyear'][i])

    for i in mainJson['weekday']:
        weekday.append(mainJson['weekday'][i])

    for i in mainJson['hour']:
        hour.append(mainJson['hour'][i])


    #append to dict, for making dataframe

    mainDict = {}
    mainDict['Dayofyear']= mainDayofyear
    mainDict['availableBikes']= available_bikes
    mainDict['weekday']= weekday
    mainDict['hour']=hour

    #change types for join
    mainDf = pd.DataFrame.from_dict(mainDict)
    mainDf['Dayofyear']=mainDf['Dayofyear'].astype(float)
    mainDf.drop_duplicates()
    mainDf.reset_index(drop=True)
    mainDf.dtypes

    # drop duplicates that share an hour and day 
    weatherDf=pd.DataFrame.from_dict(weatherDict)
    weatherDf.drop_duplicates()
    weatherDf= weatherDf.drop_duplicates(['hour','Dayofyear'])

    #change types for merging the two tables
    weatherDf['Dayofyear']=weatherDf['Dayofyear'].astype(str)
    weatherDf['hour']=weatherDf['hour'].astype(str)
    weatherDf['period'] = weatherDf[['Dayofyear', 'hour']].apply(lambda x: ''.join(x), axis=1)

    #change data types to allow merging tables
    mainDf['hour']=mainDf['hour'].astype(float)
    mainDf.head()
    mainDf['Dayofyear']=mainDf['Dayofyear'].astype(str)
    mainDf['hour']=mainDf['hour'].astype(str)
    mainDf['period'] = mainDf[['Dayofyear', 'hour']].apply(lambda x: ''.join(x), axis=1)

    newMain= mainDf.groupby([mainDf["Dayofyear"],mainDf['period'],mainDf['weekday'], mainDf["hour"]]).mean().round()

    newMain.reset_index(inplace=True)

    # merge tables
    newdf= pd.merge(newMain, weatherDf, how='right', left_on='period', right_on = 'period')

    # drop duplicate columns and period column
    newdf = newdf.drop('hour_y', 1)
    newdf = newdf.drop('period', 1)
    newdf = newdf.drop('Dayofyear_y', 1)

    # tidy up data types and names
    newdf['dayOfYear']=newdf['Dayofyear_x'].astype(float)
    newdf['availableBikes']=newdf['availableBikes'].astype(float)
    newdf['hour']=newdf['hour_x'].astype(float)
    newdf = newdf.drop('hour_x', 1)
    newdf = newdf.drop('Dayofyear_x', 1)
    newdf.head()

    # remove all NaN columsn from target feature
    newdf = newdf[np.isfinite(newdf['availableBikes'])]

    # set categorical features
    newdf['description'].astype('category')
    newdf.dtypes
    newdf = newdf[newdf.description !='light shower snow']
    newdf = newdf[newdf.description !='description_light snow']
    
    #Start training Model

    # decided not to use dayofyear as a continuous feature
    df_cont_feat = newdf[['hour']]
    df_dummies_weekday = pd.get_dummies(newdf[['weekday']])
    df_dummies_weather=pd.get_dummies(newdf[['description']])

    # Add dummies to the other continuous features
    X = pd.concat([df_cont_feat, df_dummies_weekday, df_dummies_weather], axis =1)
    y = newdf[['availableBikes']]

    # Train RF with 100 trees
    rfc = RandomForestClassifier(n_estimators=1000, max_features='auto', oob_score=True, random_state=1)
    # Fit model on full dataset
    rfc.fit(X, y)

    return joblib.dump(rfc, 'static/'stationNumber+'prediction.pkl')

In [73]:
predictionWeather = requests.get("http://localhost:5000/weather")

predictionWeather = json.loads(predictionWeather.text)

for i in predictionWeather['weatherlist']:
    forecastDescription.append(i['description'])
    forecastDescription.append(i['description'])
    forecastDescription.append(i['description'])
for i in predictionWeather['weatherlist']:
    dt.append(int(i['dt']))
    dt.append(int(i['dt'])+3600)
    dt.append(int(i['dt'])+7200)
predictionWeather={}
predictionWeather['dt']=dt
predictionWeather['description']=forecastDescription

forecastDf = pd.DataFrame.from_dict(predictionWeather)
forecastDf.head()

forecastDf['dt'] =  pd.to_datetime((forecastDf['dt']), unit='s')
forecastDf['weekday'] = forecastDf['dt'].dt.weekday_name
forecastDf['hour'] = forecastDf['dt'].dt.hour
forecastDf.drop('dt', axis=1, inplace=True)
forecastDf.head()

# Prepare the data, turn categorical feature  into dummies.
# decided not to use dayofyear as a continuous feature
prediction_dummies_weekday = pd.get_dummies(forecastDf[['weekday']])
prediction_dummies_weather=pd.get_dummies(forecastDf[['description']])

currentOrder = prediction_dummies_weekday.columns.tolist()
idealOrder = df_dummies_weekday.columns.tolist()
for i in idealOrder:
    if i not in currentOrder:
        prediction_dummies_weekday[i]=0
prediction_dummies_weekday= prediction_dummies_weekday[idealOrder]


currentWeather=prediction_dummies_weather.columns.tolist()
idealWeather= df_dummies_weather.columns.tolist()
for i in idealWeather:
    if i not in currentWeather:
        prediction_dummies_weather[i]=0
prediction_dummies_weather=prediction_dummies_weather[idealWeather]

forecast = pd.concat([forecastDf['hour'], prediction_dummies_weekday, prediction_dummies_weather], axis =1)


/Users/benjaminhowarth/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]


In [ ]:
clf = joblib.load('forecastModel.pkl')

In [ ]:
clf.predict(forecast)